In [1]:
# Cài đặt lại protobuf để sửa lỗi xung đột với TensorFlow trên Kaggle
!pip install protobuf==3.20.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tens

# Khai báo thư viện & Cấu hình

In [2]:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
import gc

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Cấu hình đường dẫn (Trỏ vào thư mục LSTM)
BASE_PATH = '/kaggle/input/preprocessed-hong-kong-pollutant-dataset/processed/lstm_sequences'
TARGET_COL = 'target_value'  # Tên cột mục tiêu trong file parquet

2025-12-03 01:05:20.618699: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764723920.973348      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764723921.086757      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Hàm đọc dữ liệu

In [3]:
def load_data_from_folder(base_path, split_name):
    """Đọc tất cả file part-*.parquet trong thư mục"""
    path_pattern = os.path.join(base_path, split_name, "part-*")
    all_files = glob.glob(path_pattern)
    
    # Lọc file hợp lệ
    data_files = [f for f in all_files if "part-" in f and not f.endswith(".crc")]
    data_files.sort()
    
    print(f"--> Đang xử lý {len(data_files)} file trong '{split_name}'...")
    
    df_list = []
    for f in data_files:
        try:
            # Ưu tiên đọc parquet
            if f.endswith('.parquet'):
                df = pd.read_parquet(f)
            else:
                df = pd.read_csv(f)
            df_list.append(df)
        except Exception as e:
            print(f"  ❌ Lỗi file {os.path.basename(f)}: {e}")

    if not df_list:
        print("⚠️ Không tìm thấy dữ liệu!")
        return None
        
    # Gộp các file lại
    df = pd.concat(df_list, ignore_index=True)
    return df

# Hàm xử lý Sequence

In [4]:
def prepare_3d_data(df, target_col):
    """
    Phiên bản tối ưu bộ nhớ: Sử dụng float32
    """
    # 1. Tách nhãn (y)
    y = df[target_col].values.astype(np.float32) # <--- Ép kiểu float32
    
    # 2. Tìm các cột feature
    feature_cols = [c for c in df.columns if c.endswith('_sequence')]
    
    # 3. Xếp chồng các mảng lại
    x_list = []
    for col in feature_cols:
        # Chuyển list of arrays thành matrix
        col_data = np.stack(df[col].tolist())
        x_list.append(col_data)
    
    # Gộp lại và ép kiểu float32 ngay lập tức
    X = np.stack(x_list, axis=2).astype(np.float32) # <--- QUAN TRỌNG: Giảm 50% RAM
    
    return X, y, feature_cols

# Thực thi Load & Xử lý

In [5]:
import gc # Thư viện dọn rác bộ nhớ

# --- GIAI ĐOẠN 1: HUẤN LUYỆN ---

# 1. Chỉ Load Train và Val (KHÔNG LOAD TEST LÚC NÀY)
print("Step 1: Loading Train & Val data...")
df_train = load_data_from_folder(BASE_PATH, 'train')
df_val   = load_data_from_folder(BASE_PATH, 'val')

# 2. Convert sang 3D (Float32)
print("Step 2: Converting to 3D Tensor...")
X_train, y_train, feats = prepare_3d_data(df_train, TARGET_COL)
X_val, y_val, _         = prepare_3d_data(df_val, TARGET_COL)

# Lấy thông số shape
n_timesteps = X_train.shape[1]
n_features = X_train.shape[2]
print(f"Structure: {X_train.shape}")

# Xóa dataframe gốc để tiết kiệm RAM ngay lập tức
del df_train, df_val
gc.collect()

# 3. Xây dựng Model
print("Step 3: Training Model...")
model = Sequential()
model.add(Input(shape=(n_timesteps, n_features)))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')

# 4. Train
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[early_stop],
    verbose=1
)

# 5. DỌN DẸP BỘ NHỚ CỰC MẠNH (QUAN TRỌNG NHẤT)
print("Step 4: Cleaning up memory...")
# Xóa sạch dữ liệu Train/Val khỏi RAM
del X_train, y_train, X_val, y_val
# Ép dọn rác
gc.collect()
print("🧹 RAM đã được giải phóng! Sẵn sàng cho bước Test.")

Step 1: Loading Train & Val data...
--> Đang xử lý 8 file trong 'train'...
--> Đang xử lý 7 file trong 'val'...
Step 2: Converting to 3D Tensor...
Structure: (201482, 24, 18)
Step 3: Training Model...


I0000 00:00:1764723964.512055      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1764723964.512672      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Epoch 1/50


I0000 00:00:1764723970.127283     127 cuda_dnn.cc:529] Loaded cuDNN version 90300


3149/3149 ━━━━━━━━━━━━━━━━━━━━ 26s 7ms/step - loss: 0.0116 - val_loss: 0.0027
Epoch 2/50
3149/3149 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 0.0030 - val_loss: 0.0024
Epoch 3/50
3149/3149 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 0.0026 - val_loss: 0.0024
Epoch 4/50
3149/3149 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 0.0026 - val_loss: 0.0023
Epoch 5/50
3149/3149 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 0.0025 - val_loss: 0.0024
Epoch 6/50
3149/3149 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 0.0025 - val_loss: 0.0023
Epoch 7/50
3149/3149 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 0.0024 - val_loss: 0.0023
Epoch 8/50
3149/3149 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 0.0024 - val_loss: 0.0023
Epoch 9/50
3149/3149 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 0.0024 - val_loss: 0.0023
Epoch 10/50
3149/3149 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 0.0024 - val_loss: 0.0024
Epoch 11/50
3149/3149 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 0.0024 - val_loss: 0.0023
Epoch 12/50
3149/3149 ━━━━━━━━

# Đánh giá

In [6]:
# --- GIAI ĐOẠN 2: DỰ BÁO ---

# 1. Bây giờ mới load tập Test
print("Step 5: Loading Test data...")
df_test = load_data_from_folder(BASE_PATH, 'test')

# 2. Convert Test sang 3D
X_test, y_test, _ = prepare_3d_data(df_test, TARGET_COL)
del df_test # Xóa luôn df gốc
gc.collect()

# 3. Dự báo (Batch size lớn để chạy nhanh hơn nhưng vẫn an toàn)
print("Step 6: Predicting...")
y_pred = model.predict(X_test, batch_size=128, verbose=1)

# 4. Đánh giá
def evaluate(y_true, y_pred, name):
    r2 = r2_score(y_true, y_pred)
    
    # Tính MSE
    mse = mean_squared_error(y_true, y_pred)
    
    # Tính RMSE (có thể lấy căn bậc hai của MSE luôn)
    rmse = np.sqrt(mse)
    
    mae = mean_absolute_error(y_true, y_pred)
    
    # Đã xóa phần tính MRE và epsilon vì không dùng nữa
    
    print(f"\n=== KẾT QUẢ {name} ===")
    print(f"R2 Score: {r2:.4f}")
    print(f"MSE:      {mse:.4f}")  # Hiển thị MSE
    print(f"RMSE:     {rmse:.4f}")
    print(f"MAE:      {mae:.4f}")

evaluate(y_test, y_pred, "LSTM Final Result")

Step 5: Loading Test data...
--> Đang xử lý 7 file trong 'test'...
Step 6: Predicting...
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step

=== KẾT QUẢ LSTM Final Result ===
R2 Score: 0.8782
MSE:      0.0024
RMSE:     0.0493
MAE:      0.0334


In [7]:
# Lưu mô hình LSTM vào file .keras (định dạng chuẩn mới của Keras)
model_name = 'lstm_pm25.keras'
model.save(model_name)

print(f"✅ Đã lưu mô hình thành công vào file: {model_name}")
print("Bạn có thể tìm thấy nó trong mục Output bên phải để tải về.")

✅ Đã lưu mô hình thành công vào file: lstm_pm25.keras
Bạn có thể tìm thấy nó trong mục Output bên phải để tải về.
